In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ================= Cell 1: 環境安裝 (黃金穩定版) =================

import os
import shutil

print("⏳ [1/4] 正在移除衝突套件...")
# 先移除可能存在的衝突版本
!pip uninstall -y torchaudio torchcodec librosa numpy

print("⏳ [2/4] 安裝系統依賴 (FFmpeg & Fonts)...")
!sudo apt-get update -qq
!sudo apt-get install -y ffmpeg fonts-noto-cjk

print("⏳ [3/4] 安裝 Python 套件 (鎖定穩定版本)...")
# 關鍵修正：
# 1. torchaudio==2.4.1: 避開 2.5+ 版的 TorchCodec 錯誤
# 2. librosa==0.9.2: 確保 Wav2Lip 能跑 (它不支援 librosa 0.10+)
# 3. numpy<2.0: 避免新版 Numpy 的兼容性問題
!pip install -q "torch>=2.4.0" "torchaudio==2.4.1" "soundfile" "demucs" "stable-ts" "gradio_client" "pydub" "yt-dlp" "opencc-python-reimplemented" "librosa==0.9.2" "opencv-python" "numpy<2.0"

print("⏳ [4/4] 準備 Wav2Lip 模型...")
# 1. Clone Wav2Lip
if not os.path.exists('/content/Wav2Lip'):
    !git clone https://github.com/Rudrabha/Wav2Lip.git

# 2. 下載模型
!mkdir -p /content/Wav2Lip/checkpoints
!mkdir -p /content/Wav2Lip/face_detection/detection/sfd

if not os.path.exists('/content/Wav2Lip/checkpoints/wav2lip_gan.pth'):
    print("   ⬇️ 下載 wav2lip_gan.pth...")
    !wget -q -c https://huggingface.co/camenduru/Wav2Lip/resolve/main/checkpoints/wav2lip_gan.pth -O /content/Wav2Lip/checkpoints/wav2lip_gan.pth

if not os.path.exists('/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth'):
    print("   ⬇️ 下載 s3fd.pth...")
    !wget -q -c "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

# 3. 再次應用 Wav2Lip 代碼修補 (確保萬無一失)
def patch_code(file_path, old_str, new_str):
    if not os.path.exists(file_path): return
    with open(file_path, 'r') as f: content = f.read()
    if old_str in content:
        content = content.replace(old_str, new_str)
        with open(file_path, 'w') as f: f.write(content)
        print(f"   🔧 已修補: {os.path.basename(file_path)}")

w2l_path = "/content/Wav2Lip"
patch_code(f"{w2l_path}/audio.py", "librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=80)", "librosa.filters.mel(sr=hp.sample_rate, n_fft=hp.n_fft, n_mels=80)")
patch_code(f"{w2l_path}/audio.py", "np.int", "int")
patch_code(f"{w2l_path}/audio.py", "np.float", "float")
patch_code(f"{w2l_path}/inference.py", "np.int", "int")
patch_code(f"{w2l_path}/inference.py", "np.float", "float")
patch_code(f"{w2l_path}/inference.py", "img = cv2.resize(window, (img_size, img_size))", "img = cv2.resize(window, (img_size, img_size), interpolation=cv2.INTER_LANCZOS4)")

print("⏳ 掛載 Google Drive...")
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

print("\n✅ 環境安裝完成！")
print("⚠️ 請務必點擊上方選單的 [Runtime] -> [Restart Session]，然後再執行 Cell 2 和 Cell 3！")

⏳ [1/4] 正在移除衝突套件...
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: librosa 0.11.0
Uninstalling librosa-0.11.0:
  Successfully uninstalled librosa-0.11.0
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
⏳ [2/4] 安裝系統依賴 (FFmpeg & Fonts)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  fonts-noto-cjk-extra
The following NEW packages will be installed:
  fonts-noto-cjk
0 upgraded, 1 newly installed, 0 to remove and 48 not upgraded.
Need to get 61.2 MB of archives.
After this operation, 93.2 MB o

In [ ]:
# ================= Cell 2: 工具函數定義 =================

import datetime
import stable_whisper
import os

# 確保 OpenCC 存在
try:
    from opencc import OpenCC
except ImportError:
    os.system("pip install opencc-python-reimplemented")
    from opencc import OpenCC

# 時間格式化工具
def format_time(seconds):
    dt = datetime.datetime.utcfromtimestamp(seconds)
    return dt.strftime('%-H:%M:%S.%f')[:-4]

# 1. 雙軌重組邏輯
def realign_dual_tracks(original_result, user_lyrics_str, user_soramimi_str):
    print("   🧩 正在計算雙軌歌詞對齊...")
    all_words = []
    for seg in original_result.segments:
        all_words.extend(seg.words)

    target_lines = [line.strip() for line in user_lyrics_str.strip().split('\n') if line.strip()]
    soramimi_lines = [line.strip() for line in user_soramimi_str.strip().split('\n') if line.strip()]

    min_len = min(len(target_lines), len(soramimi_lines))
    new_segments = []
    word_index = 0

    for i in range(min_len):
        line_text = target_lines[i]
        sora_text = soramimi_lines[i]
        current_segment_words = []
        clean_line_len = len(line_text.replace(" ", ""))
        grabbed_chars = 0

        while grabbed_chars < clean_line_len and word_index < len(all_words):
            word_obj = all_words[word_index]
            current_segment_words.append(word_obj)
            grabbed_chars += len(word_obj.word.strip())
            word_index += 1

        if current_segment_words:
            new_segments.append({
                "start": current_segment_words[0].start,
                "end": current_segment_words[-1].end,
                "words": current_segment_words,
                "text_main": line_text,
                "text_sub": sora_text
            })
    return new_segments

# 2. 生成雙軌 ASS 字幕檔
def generate_ktv_ass_dual(segments, ass_path, font_name="Noto Sans CJK TC"):
    header = f"""[Script Info]
ScriptType: v4.00+
PlayResX: 1920
PlayResY: 1080

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: MainStyle,{font_name},60,&H00FFFFFF,&H000000FF,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,2,0,2,10,10,65,1
Style: SubStyle,{font_name},40,&H0000FFFF,&H000000FF,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,2,0,2,10,10,15,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
    events = []
    cc = OpenCC('s2t')

    for seg in segments:
        start_time = format_time(seg['start'])
        end_time = format_time(seg['end'])

        # --- 原詞 ---
        k_text_main = ""
        for i, word in enumerate(seg['words']):
            duration = int((word.end - word.start) * 100)
            raw_word = word.word
            has_chinese = any('\u4e00' <= char <= '\u9fff' for char in raw_word)

            if has_chinese:
                word_text = cc.convert(raw_word.strip())
            else:
                clean_word = raw_word.strip()
                if i == 0: word_text = clean_word
                else: word_text = " " + clean_word

            if word_text:
                k_text_main += f"{{\\k{duration}}}{word_text}"

        if k_text_main:
            events.append(f"Dialogue: 0,{start_time},{end_time},MainStyle,,0,0,0,,{{\\df2}}{k_text_main}")

        # --- 空耳 ---
        sora_text = seg['text_sub']
        if sora_text:
            total_duration = int((seg['end'] - seg['start']) * 100)
            events.append(f"Dialogue: 0,{start_time},{end_time},SubStyle,,0,0,0,,{{\\df2}}{{\\k{total_duration}}}{sora_text}")

    with open(ass_path, "w", encoding='utf-8') as f:
        f.write(header + "\n".join(events))
    print(f"   ✨ 雙軌 ASS 字幕檔已生成: {ass_path}")

print("✅ 工具函數加載完畢！")

✅ 工具函數加載完畢！


In [ ]:
# ================= Cell 3: 執行生成流水線 (參數順序修正版) =================

import os
import shutil
import subprocess
import torch
import stable_whisper
import sys
from pathlib import Path
from gradio_client import Client, handle_file
from pydub import AudioSegment
from demucs.separate import main as demucs_main

# 再次確認 Numpy 補丁
import numpy as np
if not hasattr(np, 'float'): np.float = float
if not hasattr(np, 'int'): np.int = int

# ================= 1. 參數設定區 =================
input_original_song = "/content/drive/MyDrive/demo/demo.mp3"
input_user_cover    = "/content/drive/MyDrive/demo/demo.mp3"
image_input_path    = "/content/drive/MyDrive/demo/桶.jpg"

model_pth   = "/content/drive/MyDrive/demo/model.pth"
model_index = "/content/drive/MyDrive/demo/model.index"
f0_change   = 0

lyrics_input = """
The snow glows white on the mountain tonight
Not a footprint to be seen
A kingdom of isolation
and it looks like I'm the queen
The wind is howling like this swirling storm inside
Couldn't keep it in
Heaven knows I've tried
"""

soramimi_input = """
特斯諾 哥羅斯 外特 噢恩 德 馬恩廷 托奈特
諾 诶 馮特普 入 唐 貝 宋
啊 金德姆 哦夫 依瑟拉申
安 依特 花格 萊克 爱姆 德 奎因
德 威恩 德 瑙林 萊克 息 斯沃林 斯托姆 内萨德
扣得 肯 爱特 因
海文 奈兹 爱弗 特赖德
"""

lyrics_language = 'en'
output_dir = "/content/drive/MyDrive/RVC轉換/Final_Ultimate_Output"

# ==========================================================

os.makedirs(output_dir, exist_ok=True)
temp_dir = Path("/content/temp_dual_process")
if temp_dir.exists(): shutil.rmtree(temp_dir)
temp_dir.mkdir(parents=True, exist_ok=True)

input_original_song = os.path.abspath(input_original_song)
input_user_cover = os.path.abspath(input_user_cover)

%cd /content/Wav2Lip
song_name = Path(input_user_cover).stem
print(f"🚀 終極流水線啟動！目標歌曲：{song_name}")


# --- 階段 1: 音源分離 (Demucs) ---
print("\n=== [1/5] 音源分離處理 ===")
def run_demucs_safe(input_file, output_path):
    print(f"   ✂️ 處理音訊: {os.path.basename(input_file)}...")
    temp_wav = temp_dir / (Path(input_file).stem + "_clean.wav")
    AudioSegment.from_file(input_file).export(temp_wav, format="wav")
    args = ["-n", "htdemucs", "--two-stems=vocals", "-o", str(output_path), str(temp_wav)]
    try:
        demucs_main(args)
    except Exception as e:
        print(f"   ⚠️ GPU 模式失敗，嘗試切換至 CPU...")
        args.extend(["-d", "cpu"])
        demucs_main(args)

try:
    run_demucs_safe(input_original_song, temp_dir / "bgm_source")
    bgm_stem = Path(input_original_song).stem + "_clean"
    final_inst_path = temp_dir / "bgm_source" / "htdemucs" / bgm_stem / "no_vocals.wav"

    run_demucs_safe(input_user_cover, temp_dir / "vocal_source")
    vocal_stem = Path(input_user_cover).stem + "_clean"
    user_vocals_path = temp_dir / "vocal_source" / "htdemucs" / vocal_stem / "vocals.wav"

    if not final_inst_path.exists() or not user_vocals_path.exists():
        raise FileNotFoundError("分離檔案未生成")
    print("      ✅ 音源分離完成")
except Exception as e:
    print(f"❌ Demucs 錯誤: {e}")
    raise e


# --- 階段 2: AI 變聲 (RVC) ---
print("\n=== [2/5] AI 變聲轉換 (RVC API - 順序修正版) ===")
try:
    client = Client("r3gm/RVC_ZERO")

    # 建立 Handle
    audio_handle = handle_file(str(user_vocals_path))
    model_handle = handle_file(model_pth)
    index_handle = handle_file(model_index)

    print("   📡 發送 API 請求...")

    # ⚠️【關鍵修正】依照錯誤訊息調整順序
    # 根據 Error: Parameter 3 expects Method (rmvpe+), not Index File.
    # 新順序預測: [Audio], Model, Method, Pitch, Index ...
    result = client.predict(
        [audio_handle],      # 1. 音訊 (必須是 List)
        model_handle,        # 2. 模型 (.pth)
        "rmvpe+",            # 3. 演算法 (原本錯放成 index, 現在移到這裡)
        f0_change,           # 4. 變調 (int)
        index_handle,        # 5. Index 檔案 (原本在第3位, 現在移到這裡)
        0.75,                # 6. Index Rate
        3,                   # 7. Filter Radius
        0,                   # 8. Resample
        0.25,                # 9. RMS Mix
        0.33,                # 10. Protect
        api_name="/run"
    )

    # 結果處理
    converted_vocal_path = result[0] if isinstance(result, (list, tuple)) else result
    print(f"      ✅ 變聲成功: {converted_vocal_path}")

except Exception as e:
    print(f"❌ RVC 錯誤: {e}")
    print("💡 若這次還是參數錯誤，我們將需要執行「API 診斷腳本」來查看最正確的參數表。")
    raise e


# --- 階段 3: 混音 ---
print("\n=== [3/5] 混音 ===")
voc_audio = AudioSegment.from_file(converted_vocal_path)
bgm_audio = AudioSegment.from_file(str(final_inst_path))
final_audio = voc_audio.overlay(bgm_audio)
ai_cover_path = os.path.join(output_dir, f"{song_name}_Godtone_Ver.wav")
final_audio.export(ai_cover_path, format="wav")
print(f"   💾 混音完成: {ai_cover_path}")


# --- 階段 4: Wav2Lip ---
print("\n=== [4/5] 視覺生成 (Wav2Lip) ===")
lipsync_video_path = os.path.join(output_dir, f"{song_name}_lipsync.mp4")
checkpoint_path = "checkpoints/wav2lip_gan.pth"

if not os.path.exists(lipsync_video_path):
    print("   ⏳ 正在生成嘴型影片...")
    cmd = f"python inference.py --checkpoint_path {checkpoint_path} --face \"{image_input_path}\" --audio \"{ai_cover_path}\" --outfile \"{lipsync_video_path}\" --resize_factor 1"
    subprocess.run(cmd, shell=True, check=True)
    print("      ✅ 影片生成完畢")
else:
    print("   ⚠️ 影片已存在，跳過。")


# --- 階段 5: 字幕與合成 ---
print("\n=== [5/5] 字幕與最終合成 ===")
ass_path = os.path.join(output_dir, f"{song_name}.ass")
final_video_path = os.path.join(output_dir, f"{song_name}_Final_Ultimate.mp4")

model = stable_whisper.load_model('medium', device="cuda" if torch.cuda.is_available() else "cpu")
if lyrics_input.strip():
    print(f"   🎯 對齊歌詞...")
    result = model.align(ai_cover_path, text=lyrics_input.strip(), language=lyrics_language)
    try:
        final_segments = realign_dual_tracks(result, lyrics_input.strip(), soramimi_input.strip())
        generate_ktv_ass_dual(final_segments, ass_path)
    except NameError:
        print("❌ 警告: 找不到工具函數，請確認 Cell 2 已執行。")

print("   🎬 最終渲染...")
temp_vid, temp_ass = "temp_src.mp4", "temp_sub.ass"
shutil.copy(lipsync_video_path, temp_vid)
shutil.copy(ass_path, temp_ass)

ffmpeg_cmd = [
    "ffmpeg", "-y", "-v", "error",
    "-i", temp_vid, "-i", ai_cover_path,
    "-vf", f"ass={temp_ass}",
    "-map", "0:v", "-map", "1:a",
    "-c:v", "libx264", "-preset", "medium", "-crf", "18",
    "-c:a", "aac", "-b:a", "192k", "-shortest",
    final_video_path
]

try:
    subprocess.run(ffmpeg_cmd, check=True)
    print(f"\n🎉🎉🎉 恭喜！檔案路徑: {final_video_path}")
except Exception as e:
    print(f"❌ 合成失敗: {e}")

if os.path.exists(temp_vid): os.remove(temp_vid)
if os.path.exists(temp_ass): os.remove(temp_ass)

/content/Wav2Lip
🚀 終極流水線啟動！目標歌曲：demo

=== [1/5] 音源分離處理 ===
   ✂️ 處理音訊: demo.mp3...
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/temp_dual_process/bgm_source/htdemucs
Separating track /content/temp_dual_process/demo_clean.wav


100%|██████████████████████████████████████████████| 35.099999999999994/35.099999999999994 [00:01<00:00, 18.95seconds/s]


   ✂️ 處理音訊: demo.mp3...
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/temp_dual_process/vocal_source/htdemucs
Separating track /content/temp_dual_process/demo_clean.wav


100%|██████████████████████████████████████████████| 35.099999999999994/35.099999999999994 [00:01<00:00, 20.39seconds/s]


      ✅ 音源分離完成

=== [2/5] AI 變聲轉換 (RVC API - 順序修正版) ===
Loaded as API: https://r3gm-rvc-zero.hf.space ✔
   📡 發送 API 請求...
      ✅ 變聲成功: /tmp/gradio/ddbf541dac97902c6f24a85f817f4b8e755b4832e98e8d9f749a4b96d2d3cd3f/vocals_edited_noisereduce.wav

=== [3/5] 混音 ===
   💾 混音完成: /content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_Godtone_Ver.wav

=== [4/5] 視覺生成 (Wav2Lip) ===
   ⏳ 正在生成嘴型影片...
      ✅ 影片生成完畢

=== [5/5] 字幕與最終合成 ===
   🎯 對齊歌詞...


Align: 100%|██████████| 29.86/29.86 [00:00<00:00, 52.96sec/s]
Adjustment: 100%|██████████| 27.76/27.76 [00:00<00:00, 14362.14sec/s]
/tmp/ipython-input-3555755754.py:16: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.datetime.utcfromtimestamp(seconds)


   🧩 正在計算雙軌歌詞對齊...
   ✨ 雙軌 ASS 字幕檔已生成: /content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo.ass
   🎬 最終渲染...

🎉🎉🎉 恭喜！檔案路徑: /content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_Final_Ultimate.mp4
